<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/EEC_with_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip

--2019-02-28 01:32:06--  http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102568 (100K) [application/zip]
Saving to: ‘Equity-Evaluation-Corpus.zip’

Equity-Evaluation-C 100%[===================>] 100.16K   575KB/s    in 0.2s    

2019-02-28 01:32:06 (575 KB/s) - ‘Equity-Evaluation-Corpus.zip’ saved [102568/102568]



In [0]:
!unzip Equity-Evaluation-Corpus.zip

Archive:  Equity-Evaluation-Corpus.zip
  inflating: Equity-Evaluation-Corpus.csv  
  inflating: README.txt              


In [0]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)

In [0]:
EEC[0:5]


,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


# Load GloVe embeddings

In [0]:
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

Downloading...
From: https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
To: /content/gensim_glove.840B.300d.txt.bin
2.65GB [00:49, 53.4MB/s]


# Load Word2vec embeddings
CN performed better using word2vec embeddings. For all 4 sentiments, CN showed improvements on debiasing gender bias.
If using GloVe embeddings, CN showed improvements on anger, fear and joy. For the sentiment 'sadness', CN gave a higher gender difference. However, by my observations, results can be improved if using more female and male names to find conceptors. So with a larger name datasets, the results of 'sadness' may also show improvements on debiasing.

In [0]:
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:39, 42.1MB/s]


In [0]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

# Load fasttext

In [0]:
!gdown https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh

Downloading...
From: https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh
To: /content/fasttext.bin
2.42GB [00:42, 56.4MB/s]


In [0]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

fasttext = KeyedVectors.load_word2vec_format('/content/' + 'fasttext.bin', binary=True)
print('The fasttext embedding has been loaded!')

The fasttext embedding has been loaded!


# Load male and female name lists


In [0]:
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt

--2019-02-28 01:38:36--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20466 (20K) [text/plain]
Saving to: ‘male.txt’

male.txt            100%[===================>]  19.99K  --.-KB/s    in 0s      

2019-02-28 01:38:36 (241 MB/s) - ‘male.txt’ saved [20466/20466]

--2019-02-28 01:38:37--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt’

female.txt          100%[===================>]  34.91K  --.-KB/s    in 0.07s   

2019-02-28 01:38:37 (469 KB/s) - ‘female.txt’ saved [35751/35751]



In [0]:
!wget https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv

--2019-02-28 02:52:14--  https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921600 (900K) [text/plain]
Saving to: ‘Black-Male-Names.csv’

Black-Male-Names.cs 100%[===================>] 900.00K  --.-KB/s    in 0.04s   

2019-02-28 02:52:14 (20.8 MB/s) - ‘Black-Male-Names.csv’ saved [921600/921600]



In [0]:
!wget https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv

--2019-02-28 02:52:44--  https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1149042 (1.1M) [text/plain]
Saving to: ‘White-Male-Names.csv’

White-Male-Names.cs 100%[===================>]   1.10M  --.-KB/s    in 0.04s   

2019-02-28 02:52:44 (24.9 MB/s) - ‘White-Male-Names.csv’ saved [1149042/1149042]



In [0]:
!wget https://gist.githubusercontent.com/mbejda/26ad0574eda7fca78573/raw/6936d1a8f5fa5220f2f60a51a06a35b172c50f93/White-Female-Names.csv

--2019-02-28 03:08:45--  https://gist.githubusercontent.com/mbejda/26ad0574eda7fca78573/raw/6936d1a8f5fa5220f2f60a51a06a35b172c50f93/White-Female-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121903 (119K) [text/plain]
Saving to: ‘White-Female-Names.csv’

White-Female-Names. 100%[===================>] 119.05K  --.-KB/s    in 0.02s   

2019-02-28 03:08:45 (5.25 MB/s) - ‘White-Female-Names.csv’ saved [121903/121903]



In [0]:
!wget https://gist.githubusercontent.com/mbejda/9dc89056005a689a6456/raw/bb6ef2375f1289d0ef10dbd8e9469670ac23ceab/Black-Female-Names.csv

--2019-02-28 03:08:49--  https://gist.githubusercontent.com/mbejda/9dc89056005a689a6456/raw/bb6ef2375f1289d0ef10dbd8e9469670ac23ceab/Black-Female-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64757 (63K) [text/plain]
Saving to: ‘Black-Female-Names.csv’

Black-Female-Names. 100%[===================>]  63.24K  --.-KB/s    in 0.01s   

2019-02-28 03:08:50 (4.26 MB/s) - ‘Black-Female-Names.csv’ saved [64757/64757]



In [0]:
glove = KeyedVectors.load_word2vec_format('/content/' + 'gensim_glove.840B.300d.txt.bin', binary=True)

In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/' + 'GoogleNews-vectors-negative300.bin', binary=True)

# Get frequency list (used in calculating sentence embeddings)

In [0]:
!git clone https://github.com/PrincetonML/SIF

fatal: destination path 'SIF' already exists and is not an empty directory.


In [0]:
wikiWordsPath = '/content' + '/SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

frequencies = {}
with open(wikiWordsPath, "r+") as f_in:
    for line in f_in:
        frequencies[line.split(' ')[0]] = float(line.split(' ')[1])

# Load white/black names

In [0]:
word_list = []

In [0]:

f = open('/content/White-Male-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [0]:
len(word_list)

79129

In [0]:
f = open('/content/Black-Male-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [0]:
len(word_list)

79129

In [0]:
f = open('/content/White-Female-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [0]:
f = open('/content/Black-Female-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [0]:
len(word_list)

7038

# Load gender names

In [0]:
#load names from larger name dataset
f = open('/content/male.txt', 'r+')
i = 0
word_list= []
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
#load names from larger name dataset
f = open('/content/female.txt', 'r+')
i = 0
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
len(word_list)

7944

In [0]:
f = open('/content/name.txt', 'r+')
#word_list = []
for line in f:
  for item in line.rstrip().split(' '):
    word_list.append(item)
f.close()

In [0]:
word_list

# Load Racial Names

In [0]:
# These are names in the paper
word_list = ['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack','Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen']

In [0]:
EEC['Person'].unique()

array(['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'he',
       'this man', 'this boy', 'my brother', 'my son', 'my husband',
       'my boyfriend', 'my father', 'my uncle', 'my dad', 'Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen', 'she', 'this woman', 'this girl', 'my sister',
       'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
       'my mom', 'him', 'her'], dtype=object)

In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
#For word2vec data
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
cn_embeddings.shape

(7099, 300)

In [0]:
#Sentence embeddings

In [0]:
import nltk

class Sentence:
    
    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(normalized_sentence)]

In [0]:
#Raw Sentence Embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

def run_sif_benchmark(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
#    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    #average sentiment embedding based on several emotional words of the same sentiment
#    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis=0)    
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])


    return proj1, proj2

In [0]:
#CN Sentence Embeddings
def run_conceptor_benchmark(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha =2): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
 #   sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
 #   sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis = 0) 
 #   print(sen_embeddings)
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
        
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    #negC = np.eye(300) - SWC[model_str]
    
    
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])
    return proj1, proj2

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.034567342436660625 0.012273898802537408
t value and p value :
Ttest_relResult(statistic=13.849455809836892, pvalue=4.572745056474093e-35) Ttest_relResult(statistic=6.669230653088165, pvalue=1.0084986003378286e-10)
race difference on Raw and CN data:
0.024553912881999745 0.0011690208544382586
t value and p value :
Ttest_relResult(statistic=8.261900036278009, pvalue=3.0162583244598363e-15) Ttest_relResult(statistic=-0.5469156271130736, pvalue=0.5847862167151139)
race difference on Raw and CN data:
0.008068057572518187 0.0031396870145185777
t value and p value :
Ttest_relResult(statistic=5.40095963161286, pvalue=1.2286794299439083e-07) Ttest_relResult(statistic=2.350573923835871, pvalue=0.019300517055452857)
race difference on Raw and CN data:
0.028750490054312083 0.017983265350474693
t value and p value :
Ttest_relResult(statistic=19.54372892885523, pvalue=5.545094747635267e-58) Ttest_relResult(statistic=15.745114359232701, pvalue=1.4009279830328533e

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.034567342436660625 0.009929673733909112
t value and p value :
Ttest_relResult(statistic=13.849455809836892, pvalue=4.572745056474093e-35) Ttest_relResult(statistic=5.564122085509471, pvalue=5.2552721196549e-08)
race difference on Raw and CN data:
0.024553912881999745 0.004532087437763721
t value and p value :
Ttest_relResult(statistic=8.261900036278009, pvalue=3.0162583244598363e-15) Ttest_relResult(statistic=-2.2142666139122973, pvalue=0.02745618826465397)
race difference on Raw and CN data:
0.008068057572518187 0.0024199706160516954
t value and p value :
Ttest_relResult(statistic=5.40095963161286, pvalue=1.2286794299439083e-07) Ttest_relResult(statistic=1.979455935817103, pvalue=0.048550112213081746)
race difference on Raw and CN data:
0.028750490054312083 0.015889093653560168
t value and p value :
Ttest_relResult(statistic=19.54372892885523, pvalue=5.545094747635267e-58) Ttest_relResult(statistic=15.293601422475524, pvalue=9.001126995726789e-41)

In [0]:
#Add extra names and pronouns
#load names and pronouns from EEC
name_pron = list(set(EEC['Person']))
for item in name_pron:
  num = item.rstrip().split(' ')
  if len(num)!=1:
    word_list.append(num[1])
  else:
    word_list.append(num[0])

In [0]:
len(word_list)

8068

In [0]:
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='male')]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.06474567235189908 0.04209460345394559
t value and p value :
Ttest_relResult(statistic=-39.22921469721085, pvalue=5.4672329176397445e-130) Ttest_relResult(statistic=-28.314120707505623, pvalue=1.9678842743317127e-92)
race difference on Raw and CN data:
0.09000543647654273 0.057980282036428514
t value and p value :
Ttest_relResult(statistic=-53.68250229510699, pvalue=1.0070693011474893e-170) Ttest_relResult(statistic=-45.070483345589906, pvalue=1.4577893766486125e-147)
race difference on Raw and CN data:
0.07573256126403977 0.053055973290192356
t value and p value :
Ttest_relResult(statistic=-36.3533563736058, pvalue=1.0329660357653552e-120) Ttest_relResult(statistic=-33.692193529495704, pvalue=9.716275805084765e-112)
race difference on Raw and CN data:
0.0338204970991427 0.0341980605595707
t value and p value :
Ttest_relResult(statistic=-22.533353932748184, pvalue=4.8170981260922015e-70) Ttest_relResult(statistic=-26.08957574516559, pvalue=5.3332023

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='female')]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.05750085090959179 0.04480440422230783
t value and p value :
Ttest_relResult(statistic=-25.594524554962252, pvalue=4.332972426975255e-82) Ttest_relResult(statistic=-18.32019442951023, pvalue=5.245264394890672e-53)
race difference on Raw and CN data:
0.08173955082869465 0.05799314785779286
t value and p value :
Ttest_relResult(statistic=-33.008121817138765, pvalue=2.2699881351783204e-109) Ttest_relResult(statistic=-25.057504910556165, pvalue=5.264660627277734e-80)
race difference on Raw and CN data:
0.0986180293952588 0.04440255589506588
t value and p value :
Ttest_relResult(statistic=-29.708213522391834, pvalue=1.3788509973672255e-97) Ttest_relResult(statistic=-21.810645483064388, pvalue=3.782584028784904e-67)
race difference on Raw and CN data:
0.03154974838978562 0.03211823048496119
t value and p value :
Ttest_relResult(statistic=-15.01815228780253, pvalue=1.1281620467502544e-39) Ttest_relResult(statistic=-14.605255011684568, pvalue=4.903265518921

In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='male')]
  #print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.04521691518263744 0.03932572493432523
t value and p value :
Ttest_relResult(statistic=-24.943738180772687, pvalue=1.4610446201105612e-79) Ttest_relResult(statistic=-18.22503176661601, pvalue=1.2789578011826307e-52)
race difference on Raw and CN data:
0.06226927107069027 0.0579150360781525
t value and p value :
Ttest_relResult(statistic=-38.831694571338595, pvalue=9.87549110514498e-129) Ttest_relResult(statistic=-33.469088427228854, pvalue=5.71714961932729e-111)
race difference on Raw and CN data:
0.04762017509955545 0.05283733143957083
t value and p value :
Ttest_relResult(statistic=-23.820355241106842, pvalue=3.732281641473656e-75) Ttest_relResult(statistic=-22.10551130680492, pvalue=2.481160041233764e-68)
race difference on Raw and CN data:
0.03812963352164238 0.039024320354788
t value and p value :
Ttest_relResult(statistic=-19.571734768839935, pvalue=4.2670226123119386e-58) Ttest_relResult(statistic=-15.474925753857805, pvalue=1.695818864743828

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='female')]
  #print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.017832251820264967 0.014032763219116435
t value and p value :
Ttest_relResult(statistic=9.036385017234503, pvalue=1.1307572999312533e-17) Ttest_relResult(statistic=6.753846840262091, pvalue=6.03394816913532e-11)
race difference on Raw and CN data:
0.00975292484474843 0.007445103832855499
t value and p value :
Ttest_relResult(statistic=3.07329483033857, pvalue=0.0022836973755857963) Ttest_relResult(statistic=2.3653030189780466, pvalue=0.018561601120485743)
race difference on Raw and CN data:
0.02943942180953826 0.032589400394245606
t value and p value :
Ttest_relResult(statistic=-16.240468979330785, pvalue=1.4235919380780754e-44) Ttest_relResult(statistic=-17.623152206074973, pvalue=3.5790289483384098e-50)
race difference on Raw and CN data:
0.011468743619508602 0.007270165407020305
t value and p value :
Ttest_relResult(statistic=4.786355911515621, pvalue=2.5123000440865755e-06) Ttest_relResult(statistic=2.959660791098189, pvalue=0.00329022431975679

# Using averaged sentiment embeddings

In [0]:
#Raw Sentence Embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

def run_sif_benchmark(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    #average sentiment embedding based on several emotional words of the same sentiment
    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis=0)    
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(sen_embeddings)/norm(sen_embeddings)
    proj2 = embedding2.dot(sen_embeddings)/norm(sen_embeddings)


    return proj1, proj2

In [0]:
#CN Sentence Embeddings
def run_conceptor_benchmark(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha =2): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis = 0) 
 #   print(sen_embeddings)
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
        
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    #negC = np.eye(300) - SWC[model_str]
    
    
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    proj1 = embedding1.dot(sen_embeddings)/norm(sen_embeddings)
    proj2 = embedding2.dot(sen_embeddings)/norm(sen_embeddings)
    return proj1, proj2

In [0]:
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.04662572981549439 0.020522541404676367
t value and p value :
Ttest_relResult(statistic=18.876431918735776, pvalue=2.8639485874507208e-55) Ttest_relResult(statistic=10.708957974466772, pvalue=2.4976579236435985e-23)
race difference on Raw and CN data:
0.04998477386310169 0.022366878710030926
t value and p value :
Ttest_relResult(statistic=18.281446420194115, pvalue=7.540233841077412e-53) Ttest_relResult(statistic=9.122866641605519, pvalue=5.946521198149247e-18)
race difference on Raw and CN data:
0.032158537610318456 0.015093970440087463
t value and p value :
Ttest_relResult(statistic=14.240858165282138, pvalue=1.3406594428461711e-36) Ttest_relResult(statistic=7.094972375095749, pvalue=7.2563526914607235e-12)
race difference on Raw and CN data:
0.036845289669755144 0.022301669105866284
t value and p value :
Ttest_relResult(statistic=15.365000261632002, pvalue=4.667010585304408e-41) Ttest_relResult(statistic=9.472993183364068, pvalue=4.25045052668230

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.022654936036558167 0.001450832195656062
t value and p value :
Ttest_relResult(statistic=6.118688096111274, pvalue=2.5338502682791104e-09) Ttest_relResult(statistic=0.521135073553749, pvalue=0.6026031853281411)
race difference on Raw and CN data:
0.016162885661195545 0.0054722418761947115
t value and p value :
Ttest_relResult(statistic=4.115709755822177, pvalue=4.8209785226723784e-05) Ttest_relResult(statistic=-1.8135321778994684, pvalue=0.07060857039338449)
race difference on Raw and CN data:
0.006906583520517496 0.0005984480656133786
t value and p value :
Ttest_relResult(statistic=1.9404120112602472, pvalue=0.053134732784089776) Ttest_relResult(statistic=-0.21160293473467512, pvalue=0.8325403333239442)
race difference on Raw and CN data:
0.006719709066758323 0.003074133804024249
t value and p value :
Ttest_relResult(statistic=2.0707419456135723, pvalue=0.03911769868726408) Ttest_relResult(statistic=-1.1843412824527013, pvalue=0.23708393775577877)


In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.06302937205867498 0.07254997581896215
t value and p value :
Ttest_relResult(statistic=-33.357259918411145, pvalue=1.3931237562431747e-110) Ttest_relResult(statistic=-39.29094066300023, pvalue=3.4941102652479864e-130)
race difference on Raw and CN data:
0.07547536145795196 0.07282329005495813
t value and p value :
Ttest_relResult(statistic=-41.60077802961862, pvalue=2.5381494998460953e-137) Ttest_relResult(statistic=-43.0374218848139, pvalue=1.2466506338301739e-141)
race difference on Raw and CN data:
0.12927816447736087 0.08611546453223826
t value and p value :
Ttest_relResult(statistic=-48.7537846108828, pvalue=7.681814187382401e-158) Ttest_relResult(statistic=-40.89482112715392, pvalue=3.6169478117401684e-135)
race difference on Raw and CN data:
0.07036926528797599 0.06640802101071845
t value and p value :
Ttest_relResult(statistic=-40.56185716860407, pvalue=3.8254505789367065e-134) Ttest_relResult(statistic=-40.22566073031065, pvalue=4.193127515

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.04096830017299973 0.0585017183158558
t value and p value :
Ttest_relResult(statistic=-24.210265683249048, pvalue=1.0870037112432228e-76) Ttest_relResult(statistic=-24.209658230257883, pvalue=1.092996416338596e-76)
race difference on Raw and CN data:
0.05871398460064565 0.06664692066125724
t value and p value :
Ttest_relResult(statistic=-33.9296398672444, pvalue=1.4835937730648623e-112) Ttest_relResult(statistic=-29.540503472115283, pvalue=5.677199758529597e-97)
race difference on Raw and CN data:
0.12005264560672523 0.08061730147982855
t value and p value :
Ttest_relResult(statistic=-30.262349001938492, pvalue=1.3167208516196352e-99) Ttest_relResult(statistic=-27.76839953643594, pvalue=2.1854457473446765e-90)
race difference on Raw and CN data:
0.054930740051117206 0.05865356050665282
t value and p value :
Ttest_relResult(statistic=-33.26437526762301, pvalue=2.92274620423557e-110) Ttest_relResult(statistic=-26.9798608409489, pvalue=2.09991712785908

In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.08636108169380927 0.0831680891788335
t value and p value :
Ttest_relResult(statistic=-46.22573952963908, pvalue=7.524439130415276e-151) Ttest_relResult(statistic=-42.78894524234656, pvalue=6.822388265346177e-141)
race difference on Raw and CN data:
0.08247501382371565 0.07623093064691269
t value and p value :
Ttest_relResult(statistic=-40.925204647373555, pvalue=2.9183785073658076e-135) Ttest_relResult(statistic=-37.277704932338466, pvalue=9.673073185204856e-124)
race difference on Raw and CN data:
0.0881266052142327 0.08976462966970668
t value and p value :
Ttest_relResult(statistic=-43.95729469590564, pvalue=2.4441309482164621e-144) Ttest_relResult(statistic=-36.84358799927592, pvalue=2.525189955131654e-122)
race difference on Raw and CN data:
0.07361779760494179 0.07276741545335232
t value and p value :
Ttest_relResult(statistic=-44.22799176331874, pvalue=3.9702270167735006e-145) Ttest_relResult(statistic=-34.96184279192713, pvalue=4.55781490762

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('race difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


race difference on Raw and CN data:
0.038889994770878664 0.003719162069486158
t value and p value :
Ttest_relResult(statistic=12.255393635250222, pvalue=5.795978236005018e-29) Ttest_relResult(statistic=1.9190707658722383, pvalue=0.05579065224996515)
race difference on Raw and CN data:
0.025386111664488696 0.0070509051181963
t value and p value :
Ttest_relResult(statistic=7.1722054495814564, pvalue=4.445886453145004e-12) Ttest_relResult(statistic=-3.0243525116600662, pvalue=0.0026762586986232803)
race difference on Raw and CN data:
0.0001551391796906338 0.009183603829327548
t value and p value :
Ttest_relResult(statistic=-0.0560981709042368, pvalue=0.9552957027578203) Ttest_relResult(statistic=-3.6677410283232916, pvalue=0.00028284707092270366)
race difference on Raw and CN data:
0.01722416185729896 0.008748072729915667
t value and p value :
Ttest_relResult(statistic=5.408570671112199, pvalue=1.1814760544160276e-07) Ttest_relResult(statistic=-3.5325122072161403, pvalue=0.000466949128877